In [1]:
import pandas as pd

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Описание данных 

- date - дата совершения события

- event - событие

app_install – установка приложения <br>
app_start – открыть приложения<br>
register – зарегистрироваться <br>
search – перейти на страницу поиска товаров (каталог)<br>
choose_item – отправить товар в корзину<br>
tap_basket – перейти в корзину<br>
purchase – подтверждение покупки<br>

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ<br>
google_ads – реклама в Google<br>
vk_ads – реклама в ВК<br>
instagram_ads – реклама в instagram<br>
facebook_ads – реклама в facebook<br>
referal – акция «приведи друга»<br>
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  <br>
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [2]:
delivery = pd.read_csv('KC_case_data.csv')
delivery.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB


In [4]:
delivery['date'] = pd.to_datetime(delivery['date'])

### 1) MAU февраля

In [5]:
delivery[delivery['date'].dt.month == 2].query('event == "app_start"').device_id.nunique()

75032

### 2) Количество установок в январе

In [6]:
delivery[delivery['date'].dt.month == 1].query('event == "app_install"').device_id.nunique()

80297

### 3) Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?

In [7]:
# возьмем дату установки у каждого пользователя 
installed_date = delivery\
                        .query('event == "app_install"')\
                        .groupby('device_id', as_index=False)\
                        .agg({'date': 'min'})

In [8]:
installed_date.head()

,device_id,date
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31


In [9]:
purchased_date = delivery\
                        .query('event == "purchase"')\
                        .groupby('device_id', as_index=False)\
                        .agg({'date': 'min'})
purchased_date.head()

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [10]:
full = installed_date.merge(purchased_date, how='outer', on='device_id')\
        .rename(columns={'date_x': 'installed_date', 'date_y': 'purchased_date'})

In [11]:
full.head()

,device_id,installed_date,purchased_date
0,4013,2020-01-15,NaT
1,4016,2020-01-04,NaT
2,4046,2020-01-04,2020-01-04
3,4065,2020-03-19,NaT
4,4067,2020-01-31,NaT


In [12]:
full['conversion_days'] = (full.purchased_date - full.installed_date).dt.days

In [13]:
users_installed = full\
    .groupby('installed_date', as_index=False)\
    .agg(count_users_installed=('installed_date', 'count'))

In [14]:
users_purchased = full\
    .query('conversion_days <= 7')\
    .groupby('installed_date', as_index=False)\
    .agg(count_users_purchased=('conversion_days', 'count'))

In [15]:
cr = users_installed.merge(users_purchased, how='outer', on='installed_date')

In [16]:
cr['cr'] = cr.count_users_purchased / cr.count_users_installed * 100

In [17]:
cr.sort_values('cr', ascending=False)

,installed_date,count_users_installed,count_users_purchased,cr
0,2020-01-01,3579,1408,39.340598
8,2020-01-09,1424,558,39.185393
14,2020-01-15,4310,1650,38.283063
13,2020-01-14,5173,1973,38.140344
1,2020-01-02,3144,1186,37.722646
...,...,...,...,...
77,2020-03-18,1171,235,20.068318
88,2020-03-29,1117,223,19.964190
81,2020-03-22,1261,251,19.904837
87,2020-03-28,1091,209,19.156737


### 4) Укажите значение CR из предыдущего вопроса в %

In [18]:
cr.query('installed_date == "2020-01-01"').cr

0    39.340598
Name: cr, dtype: float64

### 5) С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [19]:
delivery.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [24]:
delivery.query('event == "app_install"').utm_source.value_counts()

-                32460
yandex-direct    29368
google_ads       26286
vk_ads           23189
instagram_ads    20096
facebook_ads     13916
referal           9282
Name: utm_source, dtype: int64

### 6) Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

Events:
- «Регистрация» - register
- «Поиск» - search
- «Добавление товара» - choose_item
- «Переход в корзину» - tap_basket
- «Покупка» - purchase

In [42]:
funnel = delivery[['date', 'event', 'device_id']]\
        .query('event in ("register", "search", "choose_item", "tap_basket", "purchase")')

In [43]:
registered = delivery\
            .query('event == "register"')\
            .groupby('device_id', as_index=False)\
            .agg(register_date=('date', 'min'))

In [44]:
registered.head()

,device_id,register_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4067,2020-01-31
4,4120,2020-01-07


In [45]:
funnel = funnel.merge(registered, how='right', on='device_id')

In [58]:
funnel.shape

(1403453, 4)

In [47]:
# Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 
funnel = funnel[funnel.date >= funnel.register_date]

In [48]:
funnel.head()

,date,event,device_id,register_date
13,2020-02-01,choose_item,4014,2020-02-01
14,2020-02-01,purchase,4014,2020-02-01
15,2020-02-01,search,4014,2020-02-01
16,2020-02-01,tap_basket,4014,2020-02-01
17,2020-02-03,search,4014,2020-02-01


In [55]:
registered = funnel.query('event == "register"').device_id.nunique() 
registered

78310

In [56]:
searched = funnel.query('event == "search"').device_id.nunique()
searched

78310

In [54]:
events = ("register", "search", "choose_item", "tap_basket", "purchase")

In [61]:
for i in range(1, len(events)):
    ev1 = events[i-1]
    ev2 = events[i]
    print(f"{ev1} -> {ev2}", funnel.query('event == @ev2').device_id.nunique() / funnel.query('event == @ev1').device_id.nunique())

register -> search 1.0
search -> choose_item 1.0
choose_item -> tap_basket 1.0
tap_basket -> purchase 0.9050568254373643


### 7) Пользователи, пришедшие с какого канала, показали самую низкую конверсию в первую покупку?

In [62]:
cr = delivery[['date', 'event', 'device_id', 'utm_source']]

In [63]:
sources = cr.utm_source.unique()

In [69]:
for source in sources:
    all_count = cr.query('event == "app_start" & utm_source == @source').device_id.nunique()
    purchased_count = cr.query('event == "purchase" & utm_source == @source').device_id.nunique()
    
    print(source, '\t', purchased_count / all_count)

- 	 0.3175556746001377
vk_ads 	 0.3628040728894627
referal 	 0.3995227329816629
facebook_ads 	 0.3473822090380244
google_ads 	 0.29767405229444505
instagram_ads 	 0.34665807698502177
yandex-direct 	 0.2954774363131648


### 8) Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

In [72]:
sources = delivery.utm_source.unique()

In [70]:
first_purchase = delivery.query('event == "purchase"').drop_duplicates('device_id', keep='first')

In [71]:
first_purchase.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-
7692,2020-01-01,purchase,374.0,android,77448,female,Saint-Petersburg,yandex-direct
7693,2020-01-01,purchase,98.5,ios,7458607,female,Moscow,instagram_ads
7694,2020-01-01,purchase,304.5,ios,3520947,male,Moscow,instagram_ads
7695,2020-01-01,purchase,153.5,android,1170605,male,Moscow,google_ads


In [75]:
first_purchase.groupby('utm_source').agg({'purchase_sum': 'median'}).sort_values('purchase_sum')

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


### 9) Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

Данные по затратам на рекламу:
- Яндекс – 10 491 707 руб.

- Гугл – 10 534 878 руб.

- Фейсбук – 8 590 498 руб.

- Инстаграм – 8 561 626 руб.

- ВК – 9 553 531 руб.

In [77]:
sources = delivery.utm_source.unique()

In [80]:
sources

array(['-', 'vk_ads', 'referal', 'facebook_ads', 'google_ads',
       'instagram_ads', 'yandex-direct'], dtype=object)

In [79]:
income = delivery\
    .query('event == "purchase"')\
    .groupby('utm_source', as_index=False)\
    .agg({'purchase_sum': 'sum'})

In [81]:
income

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [85]:
marketing = (-1, 8590498, 10534878, 8561626, -1, 9553531, 10491707)

In [86]:
income['marketing'] = marketing

In [87]:
income

,utm_source,purchase_sum,marketing
0,-,21449749.5,-1
1,facebook_ads,12249901.0,8590498
2,google_ads,12868276.0,10534878
3,instagram_ads,14546969.0,8561626
4,referal,8837044.5,-1
5,vk_ads,16389652.5,9553531
6,yandex-direct,13915368.0,10491707


In [89]:
income['ROMI'] = (income.purchase_sum - income.marketing) / income.marketing

In [90]:
income.sort_values('ROMI')

,utm_source,purchase_sum,marketing,ROMI
0,-,21449749.5,-1,-2.144975e+07
4,referal,8837044.5,-1,-8.837046e+06
2,google_ads,12868276.0,10534878,2.214926e-01
6,yandex-direct,13915368.0,10491707,3.263207e-01
1,facebook_ads,12249901.0,8590498,4.259826e-01
3,instagram_ads,14546969.0,8561626,6.990895e-01
5,vk_ads,16389652.5,9553531,7.155597e-01
